# Cargamos librerias y conectamos la base de datos

In [5]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import time, json, random
from random import randrange
from datetime import timedelta, datetime
from random import randint
from lorem_text import lorem
from flask import jsonify

es = Elasticsearch("http://127.0.0.1:9200")

print(f"Connected to ElasticSearch cluster `{es.info().body['cluster_name']}`")

Connected to ElasticSearch cluster `docker-cluster`


# Generación de datos aleatorios

In [6]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [7]:
d1 = datetime.strptime('1/1/1950 12:59 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2022 01:00 AM', '%m/%d/%Y %I:%M %p')
print(random_date(d1, d2))  

1979-06-10 11:11:29


In [8]:
descripciones = [
    'Transferencia interbancaria SIMPLE',
    'DEB.CTA.P/C.INTERNET',
    'Debito Cta por ACH',
    'Retiro en efectivo',
    'AB.CTA.P/MOVIL',
    'Pago en interés',
    'Debito Cta por compra',
    'Debito Cta por pago de Seguro',
    'Retención IVA'
]

# Creación de índices

In [9]:
mapping = {
    "mappings": {
        "properties": {
            "fecha": {
                "type": "date"
            },
            "agencia": {
                "type": "keyword"
            },
            "monto": {
                "type": "float"
            },
            "descripcion": {
                "type": "text"
            },
            "saldo": {
                "type": "float"
            },
            "nota": {
                "type": "text"
            },
        }
    }
}

response = es.indices.create(
    index="extracto_cuenta",
    mappings = mapping,
    ignore=400 # ignore 400 already exists code
)

/tmp/ipykernel_7182/5996740.py:26: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.indices.create(


# Aproximadamente 100 000 registros

## Carga de datos

### De forma individual

In [12]:
start_time = time.time()

for i in range(50000):
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }
    
    es.index(index="extracto_cuenta", document=insert_data)

end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

KeyboardInterrupt: 

### De forma masiva

In [45]:
def gendata():
    for i in range(50000):
        fecha = random_date(d1, d2)
        agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
        monto = round(random.uniform(10.0, 4000.0), 2)
        descripcion = random.choice(descripciones)
        saldo = round(random.uniform(0.0, 10000000.0), 2)
        nota = lorem.words(randint(1,13))

        yield {
            "_index": "extracto_cuenta",
            'fecha': fecha,
            'agencia': agencia,
            'monto': monto,
            'descripcion': descripcion,
            'saldo': saldo,
            'nota': nota
        }

start_time = time.time()
helpers.bulk(es, gendata())
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:25.162890


### Un solo dato

In [46]:
start_time = time.time()

fecha = random_date(d1, d2)
agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
monto = round(random.uniform(10.0, 4000.0), 2)
descripcion = random.choice(descripciones)
saldo = round(random.uniform(0.0, 10000000.0), 2)
nota = lorem.words(randint(1,13))

insert_data = {
    'fecha': fecha,
    'agencia': agencia,
    'monto': monto,
    'descripcion': descripcion,
    'saldo': saldo,
    'nota': nota
}

es.index(index="extracto_cuenta", document=insert_data)

end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:00.173278


In [11]:
!curl -XGET http://localhost:9200/extracto_cuenta/_count

{"count":26081,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}

## Consultas a la base de datos

### Búsqueda por fecha y agencia

In [48]:
def busqueda_fecha_agencia(month, year, agencia):
    from_date_str = f"{year}-{month}-01" + "T00:00:00+00:00"
    to_date_str = f"{year}-{month}-30" + "T23:59:59+00:00"
    query = {
            "bool": {
                "must": [
                    { "match": { "agencia": f"{agencia}" } },
                    {"range" : {
                        "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}"}}
                    }
                ]
            }
    }

    return es.search(index='extracto_cuenta', query=query)

In [49]:
start_time = time.time()
results = busqueda_fecha_agencia(month="05", year = "2021", agencia = "Santa Cruz")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 18 en un tiempo de 0:00:00.008642
{'fecha': '2021-05-30T22:10:08', 'agencia': 'Santa Cruz', 'monto': 2603.27, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 379311.91, 'nota': 'quas doloribus'}
{'fecha': '2021-05-29T08:38:42', 'agencia': 'Santa Cruz', 'monto': 3519.13, 'descripcion': 'Pago en interés', 'saldo': 6521150.3, 'nota': 'est inventore illum blanditiis qui dolorem sequi vero iusto quibusdam'}
{'fecha': '2021-05-14T21:40:03', 'agencia': 'Santa Cruz', 'monto': 252.07, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 5440489.4, 'nota': 'laborum incidunt iste quisquam iusto necessitatibus quibusdam quaerat'}
{'fecha': '2021-05-25T23:33:40', 'agencia': 'Santa Cruz', 'monto': 2820.07, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 2666885.97, 'nota': 'perferendis minima dolore'}
{'fecha': '2021-05-13T12:47:10', 'agencia': 'Santa Cruz', 'monto': 3898.0, 'descripcion': 'Pago en interés', 'saldo': 9454321.8, 'nota': 'excepturi illo placea

### Búsqueda por montos mayores a determinado número y agencia

In [50]:
def busqueda_monto_mayor(monto, agencia):
    query = {
            "bool": {
            "must": [
                { "match": 
                    { "agencia": f"{agencia}" } 
                },
                {"range" : {
                "monto": { "gte" : monto}}
                }
            ]
            }
    }

    return es.search(index='extracto_cuenta', query=query)

In [51]:
start_time = time.time()
results = busqueda_monto_mayor(monto=3000, agencia = "Cochabamba")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 3238 en un tiempo de 0:00:00.009668
{'fecha': '1978-04-23T05:53:58', 'agencia': 'Cochabamba', 'monto': 3886.08, 'descripcion': 'Pago en interés', 'saldo': 2924013.63, 'nota': 'animi perferendis deserunt voluptas vitae ullam harum'}
{'fecha': '1976-01-06T06:39:38', 'agencia': 'Cochabamba', 'monto': 3966.17, 'descripcion': 'Debito Cta por ACH', 'saldo': 3249567.24, 'nota': 'quos veritatis velit qui laboriosam consequuntur'}
{'fecha': '2021-06-19T18:15:08', 'agencia': 'Cochabamba', 'monto': 3586.27, 'descripcion': 'Retiro en efectivo', 'saldo': 3479542.93, 'nota': 'ducimus obcaecati repellendus totam reprehenderit expedita facilis ipsa quisquam iste optio tenetur praesentium'}
{'fecha': '1954-07-04T08:30:40', 'agencia': 'Cochabamba', 'monto': 3573.94, 'descripcion': 'Pago en interés', 'saldo': 438773.1, 'nota': 'vitae ratione ducimus velit architecto laborum iste doloremque adipisci ab illo nobis'}
{'fecha': '1952-09-27T03:30:21', 'agencia': 'Cochabamba', 'monto': 3954.12, 

### Búsqueda considerando varios campos

In [52]:
def busqueda_compleja(saldo_min, saldo_max, agencia1, agencia2, descripcion):
    query = {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {"match": {"agencia": f"{agencia1}"}},
                                {"match": {"agencia": f"{agencia2}"}}
                            ]
                        }
                    },
                    {
                        "bool": {
                            "must": [
                                {
                                    "range" : {
                                        "monto": { "gte" : saldo_min, "lte": saldo_max}
                                    }
                                }
                            ]
                        }
                    },
                    {
                        "match": {"descripcion": f"{descripcion}"}
                    }
                ]
            }
    }

    return es.search(index='extracto_cuenta', query=query)

In [65]:
start_time = time.time()
results = busqueda_compleja(saldo_min=1000, saldo_max=1500000, agencia1="La Paz", agencia2="Pando", descripcion="Retiro en efectivo")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 4112 en un tiempo de 0:00:00.027304
{'fecha': '1983-11-20T08:42:35', 'agencia': 'La Paz', 'monto': 3006.8, 'descripcion': 'Retiro en efectivo', 'saldo': 8128558.78, 'nota': 'nemo animi'}
{'fecha': '1959-10-06T05:50:33', 'agencia': 'La Paz', 'monto': 3741.66, 'descripcion': 'Retiro en efectivo', 'saldo': 3727318.23, 'nota': 'culpa cupiditate autem officiis impedit alias consequuntur libero numquam mollitia aliquam'}
{'fecha': '1989-02-07T00:42:37', 'agencia': 'La Paz', 'monto': 1151.8, 'descripcion': 'Retiro en efectivo', 'saldo': 1897132.92, 'nota': 'inventore quo ullam'}
{'fecha': '1961-08-09T15:37:47', 'agencia': 'La Paz', 'monto': 2767.52, 'descripcion': 'Retiro en efectivo', 'saldo': 6362562.68, 'nota': 'facilis'}
{'fecha': '1983-04-09T09:45:00', 'agencia': 'La Paz', 'monto': 1391.39, 'descripcion': 'Retiro en efectivo', 'saldo': 7085243.84, 'nota': 'ipsum facere numquam beatae nihil'}
{'fecha': '1965-11-05T00:53:48', 'agencia': 'La Paz', 'monto': 3303.3, 'descripcio

### Agregaciones: Monto promedio por agencia

In [54]:
def promedio_monto_agencia(agencia):
    aggs = {
            "aggs" : {
                    "avg" : {
                        "field" : "monto",
                    } 
            }
        }

    query = {
        "match": {"agencia": "La Paz"}
    }



    return es.search(index='extracto_cuenta', query=query, aggs=aggs)

In [55]:
start_time = time.time()
results = promedio_monto_agencia(agencia="La Paz")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

print(results['hits'])
print(results['aggregations'])

Se encontraron 10000 en un tiempo de 0:00:00.014832
{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 3.3485756, 'hits': [{'_index': 'extracto_cuenta', '_id': 'zP-8PIQBXNOVFC33WUSS', '_score': 3.3485756, '_source': {'fecha': '1956-04-09T00:21:28', 'agencia': 'La Paz', 'monto': 2871.64, 'descripcion': 'Debito Cta por compra', 'saldo': 5226693.42, 'nota': 'dolor aperiam neque obcaecati nulla doloremque unde laborum ullam non sed'}}, {'_index': 'extracto_cuenta', '_id': '0f-8PIQBXNOVFC33WUSS', '_score': 3.3485756, '_source': {'fecha': '1983-11-20T08:42:35', 'agencia': 'La Paz', 'monto': 3006.8, 'descripcion': 'Retiro en efectivo', 'saldo': 8128558.78, 'nota': 'nemo animi'}}, {'_index': 'extracto_cuenta', '_id': '3v-8PIQBXNOVFC33WUSS', '_score': 3.3485756, '_source': {'fecha': '1977-07-17T15:58:03', 'agencia': 'La Paz', 'monto': 684.3, 'descripcion': 'Retención IVA', 'saldo': 2067532.25, 'nota': 'dolores in laboriosam tenetur aspernatur fugit iusto'}}, {'_index': 'extracto_cuenta

## Actualización de datos

### Actualización de montos en un rango determinado de fechas

In [56]:
def actualizacion_fecha_monto(min_date, max_date, monto):
      from_date_str = f"{min_date}" + "T00:00:00+00:00"
      to_date_str = f"{max_date}" + "T23:59:59+00:00"
   

      script = {
            "source": f"ctx._source['monto'] = {monto}; ctx._source['nota'] = 'Actualización de monto por fecha'; ",
            "lang": "painless"
      }

      query = {
                  "range" : {
                  "fecha": { "gte" : f"{min_date}", "lte" : f"{max_date}" },
                  }
      }

      return es.update_by_query(query=query, script=script, index='extracto_cuenta')

In [66]:
start_time = time.time()
results = actualizacion_fecha_monto(min_date = "2022-01-01", max_date = "2022-05-30", monto = 7000)
end_time = time.time()


print(f"Se actualizaron {results['total']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

Se actualizaron 1 en un tiempo de 0:00:00.199623


In [58]:
query = {
    "range" : {
                  "fecha": { "gte" : f"2022-01-01", "lte" : f"2022-05-30" },
      }
  }

results = es.search(index='extracto_cuenta', query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Pago en interés', 'fecha': '2022-01-01T00:55:56', 'monto': 7000, 'saldo': 1613932.2, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
Se encontraron: 1 resultados


## Eliminar datos

### Eliminar saldos menores a determinado número

In [62]:
def eliminar_saldo_min(saldo_min):

    query = {
        "range" : {
                  "saldo": { "lte" : f"{saldo_min}"},
      }
    }

    return es.delete_by_query(index = 'extracto_cuenta', query = query)

In [63]:
start_time = time.time()
results = eliminar_saldo_min(saldo_min= 500)
end_time = time.time()

print(results)
print(f"Se eliminaron {results['total']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'took': 178, 'timed_out': False, 'total': 4, 'deleted': 4, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Se eliminaron 4 en un tiempo de 0:00:00.184631


In [64]:
query = {
    "range" : {
                  "saldo": { "lte" : 500},
      }
  }

results = es.search(index='extracto_cuenta', query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

Se encontraron: 0 resultados


# Mas de 10 millon de registros

## Carga de datos

In [13]:
def gendata():
    for i in range(500000):
        fecha = random_date(d1, d2)
        agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
        monto = round(random.uniform(10.0, 4000.0), 2)
        descripcion = random.choice(descripciones)
        saldo = round(random.uniform(0.0, 10000000.0), 2)
        nota = lorem.words(randint(1,13))

        yield {
            "_index": "extracto_cuenta",
            'fecha': fecha,
            'agencia': agencia,
            'monto': monto,
            'descripcion': descripcion,
            'saldo': saldo,
            'nota': nota
        }

start_time = time.time()

for _ in range(0,25):
    try:
        helpers.bulk(es, gendata())
    except: 
        pass
    
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 1:09:32.025914


In [7]:
!curl -XGET http://localhost:9200/extracto_cuenta/_count

{"count":10512560,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}

## Consulta de datos

### Búsqueda por fecha y agencia

In [21]:
start_time = time.time()
results = busqueda_fecha_agencia(month="05", year = "2021", agencia = "Santa Cruz")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 1512 en un tiempo de 0:00:00.044127
{'fecha': '2021-05-07T03:24:29', 'agencia': 'Santa Cruz', 'monto': 3461.86, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 9893162.25, 'nota': 'blanditiis itaque cupiditate explicabo'}
{'fecha': '2021-05-26T13:46:29', 'agencia': 'Santa Cruz', 'monto': 1247.1, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 6314865.37, 'nota': 'officiis autem adipisci officia esse corrupti'}
{'fecha': '2021-05-27T07:17:19', 'agencia': 'Santa Cruz', 'monto': 3327.88, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 5857848.61, 'nota': 'rem asperiores quisquam suscipit dicta voluptatem deleniti officia sit minus alias nisi'}
{'fecha': '2021-05-19T23:32:55', 'agencia': 'Santa Cruz', 'monto': 99.86, 'descripcion': 'Pago en interés', 'saldo': 8595914.55, 'nota': 'alias'}
{'fecha': '2021-05-19T12:24:33', 'agencia': 'Santa Cruz', 'monto': 3076.24, 'descripcion': 'Pago en interés', 'saldo': 3910177.71, 'nota': 'nam sapiente 

### Búsqueda por montos mayores a determinado número y agencia

In [22]:
start_time = time.time()
results = busqueda_monto_mayor(monto=3000, agencia = "Cochabamba")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 10000 en un tiempo de 0:00:00.065772
{'fecha': '1977-09-09T03:37:13', 'agencia': 'Cochabamba', 'monto': 3516.03, 'descripcion': 'Retiro en efectivo', 'saldo': 4780436.8, 'nota': 'dolorum aut molestiae libero adipisci modi corrupti'}
{'fecha': '2013-07-11T15:05:55', 'agencia': 'Cochabamba', 'monto': 3648.64, 'descripcion': 'Retiro en efectivo', 'saldo': 5900955.72, 'nota': 'corrupti blanditiis libero rerum sed perspiciatis nisi quisquam'}
{'fecha': '1972-12-03T17:56:35', 'agencia': 'Cochabamba', 'monto': 3413.83, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 6966391.74, 'nota': 'veniam minus sequi ullam non saepe est'}
{'fecha': '1961-12-01T15:42:51', 'agencia': 'Cochabamba', 'monto': 3424.54, 'descripcion': 'Debito Cta por compra', 'saldo': 3756687.42, 'nota': 'vitae consequatur'}
{'fecha': '2015-05-31T02:35:24', 'agencia': 'Cochabamba', 'monto': 3964.37, 'descripcion': 'Debito Cta por ACH', 'saldo': 5471718.23, 'nota': 'vero'}
{'fecha': '1980-02-27T01:22:43', 'agencia

### Búsqueda considerando varios campos

In [23]:
start_time = time.time()
results = busqueda_compleja(saldo_min=1000, saldo_max=1500000, agencia1="Beni", agencia2="Pando", descripcion="Retiro en efectivo")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 10000 en un tiempo de 0:00:00.855386
{'fecha': '1992-03-27T22:37:39', 'agencia': 'Beni', 'monto': 3514.88, 'descripcion': 'Retiro en efectivo', 'saldo': 7564017.42, 'nota': 'maiores blanditiis impedit libero laborum nostrum suscipit nobis enim corrupti dicta eveniet sit'}
{'fecha': '1955-08-11T05:50:04', 'agencia': 'Beni', 'monto': 1515.24, 'descripcion': 'Retiro en efectivo', 'saldo': 6969180.17, 'nota': 'accusamus quidem consequuntur doloremque veniam eum eveniet'}
{'fecha': '1954-07-25T13:59:31', 'agencia': 'Beni', 'monto': 2324.39, 'descripcion': 'Retiro en efectivo', 'saldo': 5438280.1, 'nota': 'sunt voluptas maxime praesentium tempora ullam nostrum sit molestias similique'}
{'fecha': '1991-12-03T16:13:37', 'agencia': 'Beni', 'monto': 2657.4, 'descripcion': 'Retiro en efectivo', 'saldo': 8994435.01, 'nota': 'molestias deserunt dignissimos et'}
{'fecha': '2007-11-12T12:23:26', 'agencia': 'Beni', 'monto': 1046.99, 'descripcion': 'Retiro en efectivo', 'saldo': 9289430.

### Agregaciones: Monto promedio por agencia

In [27]:
start_time = time.time()
results = promedio_monto_agencia(agencia="La Paz")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

print(results['hits'])
print(results['aggregations'])

Se encontraron 10000 en un tiempo de 0:00:00.487605
{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 3.340705, 'hits': [{'_index': 'extracto_cuenta', '_id': 'RVyhNYQBKdMpO4lrUdjb', '_score': 3.340705, '_source': {'fecha': '1965-10-14T12:51:17', 'agencia': 'La Paz', 'monto': 1055.86, 'descripcion': 'Retención IVA', 'saldo': 259714.1, 'nota': 'distinctio cupiditate sed impedit quasi est'}}, {'_index': 'extracto_cuenta', '_id': 'UVyhNYQBKdMpO4lrUdjb', '_score': 3.340705, '_source': {'fecha': '1972-09-17T08:22:13', 'agencia': 'La Paz', 'monto': 3804.55, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 5197980.07, 'nota': 'consectetur ipsa velit repudiandae ullam'}}, {'_index': 'extracto_cuenta', '_id': 'XVyhNYQBKdMpO4lrUdjb', '_score': 3.340705, '_source': {'fecha': '1960-08-12T05:47:43', 'agencia': 'La Paz', 'monto': 1514.74, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 6499602.55, 'nota': 'voluptates corrupti'}}, {'_index': 'extracto_cuenta', '_id': 'YVyhNYQBKdMpO4lrUdjb'

## Actualización de datos

### Actualización de montos en un rango determinado de fechas

In [32]:
start_time = time.time()
results = actualizacion_fecha_monto(min_date = "2022-01-01", max_date = "2022-05-30", monto = 7000)
end_time = time.time()


print(f"Se actualizaron {results['total']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

Se actualizaron 20 en un tiempo de 0:00:00.155452


In [33]:
query = {
    "range" : {
                  "fecha": { "gte" : f"2022-01-01", "lte" : f"2022-05-30" },
      }
  }

results = es.search(index='extracto_cuenta', query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Debito Cta por pago de Seguro', 'fecha': '2022-01-01T00:30:33', 'monto': 7000, 'saldo': 4499644.88, 'nota': 'Actualización de monto por fecha', 'agencia': 'Oruro'}
{'descripcion': 'DEB.CTA.P/C.INTERNET', 'fecha': '2022-01-01T00:11:42', 'monto': 7000, 'saldo': 5689893.75, 'nota': 'Actualización de monto por fecha', 'agencia': 'Beni'}
{'descripcion': 'Retención IVA', 'fecha': '2022-01-01T00:01:34', 'monto': 7000, 'saldo': 7351642.72, 'nota': 'Actualización de monto por fecha', 'agencia': 'Pando'}
{'descripcion': 'Retiro en efectivo', 'fecha': '2022-01-01T00:35:35', 'monto': 7000, 'saldo': 193253.51, 'nota': 'Actualización de monto por fecha', 'agencia': 'Santa Cruz'}
{'descripcion': 'Pago en interés', 'fecha': '2022-01-01T00:27:13', 'monto': 7000, 'saldo': 151798.37, 'nota': 'Actualización de monto por fecha', 'agencia': 'Cochabamba'}
{'descripcion': 'Retención IVA', 'fecha': '2022-01-01T00:49:50', 'monto': 7000, 'saldo': 5993279.21, 'nota': 'Actualización de monto por f

## Eliminar datos

### Eliminar saldos menores a determinado número

In [37]:
start_time = time.time()
results = eliminar_saldo_min(saldo_min= 1000)
end_time = time.time()

print(results)
print(f"Se eliminaron {results['total']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'took': 6926, 'timed_out': False, 'total': 554, 'deleted': 554, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Se eliminaron 554 en un tiempo de 0:00:06.930695


In [38]:
query = {
    "range" : {
                  "saldo": { "lte" : 1000},
      }
  }

results = es.search(index='extracto_cuenta', query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

Se encontraron: 0 resultados
